<a href="https://colab.research.google.com/github/Mikcl/CviTekkers/blob/main/CVI_helper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# MAKE SURE TO RUN THIS FIRST before any of the rest!!

import numpy as np
import torch
import torch.nn.functional as F
import scipy.stats as st
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.spatial.distance import cdist, pdist, squareform
from sklearn.cluster import AgglomerativeClustering
from matplotlib import pyplot as plt
import math

def element_wise_average(arr):
  return np.mean(arr, axis=0)

# SIMILARITY

# Minimise
SSD = lambda u, v: ((u-v)**2).sum()    
euclidean = lambda u, v: np.sqrt(((u-v)**2).sum())    
SAD = lambda u, v: np.absolute(u-v).sum()           

# Maximise
# requires 2*(size*size) - 1 opperations
cross_correlation = lambda u, v: np.multiply(u,v).sum()

# aka cosine of the angle
normalised_cross_correlation = lambda u, v: np.divide(   
    np.multiply(u,v).sum(),                             
    np.multiply(                                        
        np.sqrt(np.square(u).sum()),                    
        np.sqrt(np.square(v).sum())                     
    )                                                   
)

correlation_coefficient = lambda u, v: normalised_cross_correlation(
    np.subtract(u, element_wise_average(u)),
    np.subtract(v, element_wise_average(v))
)

# HELPERS

def all_similiarities(X, method):
  '''
    method - euclidean, SAD, cross_correlation, normalised_cross_correlation ...
    X - [[]] - list of features
  '''
  dm = pdist(X, method)
  return squareform(dm)

def np_square_pad(m):
  return np.pad(m, [(1, 1), (1, 1)], mode='constant', constant_values=0)

def get_matrix_patches(X,size,pad=False):
  '''
    X - [[]]
    size - n x n sized patch to create
    pad - add 0 pad if True

    returns [[[]]]  - of shape (rows,cols,size,size)
  '''
  x = X if not pad else np_square_pad(X)
  t = torch.tensor(x)
  res = t.unfold(0, size, 1).unfold(1,size,1).numpy()
  return res

(use ctrl+arrows and ctrl+shift+arrows for quick highlighting)

## RGB to Grey


In [ ]:
# Arguments - Matricies

R = [
     [208, 233,  71],
     [231, 161, 140],
     [32,  24,  245],
]

G = [
     [247 ,245 , 36],
     [40  ,124, 107],
     [248 ,204, 234],
]

B = [
     [202,  9,  173],
     [245, 217, 193],
     [167, 239, 190],
]


In [ ]:
RGB = np.array([R,G,B])
print("round to nearest int?")
element_wise_average(RGB)

## Convolution 2D

In [ ]:
# Arguments - note: the code does the rotations/flips
# make sure IMAGE is IMAGE, mask is mask 
# start with padding = 0, increment if need bigger size.
Image = [
      [0.1, 0.5, 1],
      [0, 0.1, 0.3],
      [0.4, 0.6, 0.2],  
]

mask = [
    [0.01, 0.09, 0.01],
    [0.09, 0.64, 0.09],
    [0.01, 0.09, 0.01], 
     
]
padding = 0     #

In [ ]:
def conv2d(image, mask, pad=0):
  '''
    image - [[num]]
    mask - [[num]]
  '''
  t_i = torch.tensor(image).float()
  t_m = torch.tensor(mask).flip(0).flip(1).float()
  result = F.conv2d(t_i[None, None, ...], t_m[None, None,...], padding=pad)
  return result

conv2d(Image, mask, pad=padding)

## Gaussian 2D Generation help


In [ ]:
#  G(x,y) = 1/2piσ^2 exp(- (x^2 + y^2)/ 2σ^2)
sigma = 0.6   # standard deviation
size = 3       # ideally >=6*σ , not too big for efficiency

# Calculate for x,y where x,y = 0,0 is the center
x, y = np.mgrid[-size//2 + 1:size//2 + 1, -size//2 + 1:size//2 + 1]
g = (np.exp(-((x**2 + y**2)/(2.0*sigma**2)))) / (2*np.pi*sigma**2)
g

array([[0.02768181, 0.11101489, 0.02768181],
       [0.11101489, 0.44521319, 0.11101489],
       [0.02768181, 0.11101489, 0.02768181]])

## Convolution, Number of computations


In [ ]:
#  arguments , number of ops named by function name.

mask_len = 11     # number of cells across. 
image_x = 300
image_y = 200


In [ ]:
def number_of_operations_non_separable(mask_len, width, height):
  step_one = (mask_len * mask_len) + (mask_len*mask_len -1)
  step_two = width*height*step_one
  return step_two

number_of_operations_non_separable(mask_len, image_x, image_y)


14460000

In [ ]:
def number_of_ops_separable(mask_len, width, height):
  # convolve with [mask_len x 1], then with [mask_len x 1]T
  step_one = (mask_len+mask_len -1) * width * height
  step_two = (mask_len+mask_len -1) * width * height
  result = step_one + step_two
  return result

number_of_ops_separable(mask_len, image_x, image_y)

In [ ]:
'''
Convolving the image with a DoG is equivalent to
convolving the image with a Gaussian and 
subtracting the result of convolving the image with another Gaussian.
'''
def DoG_number_of_ops_exploit(mask_len, width, height):
  one_d_conv_ops = number_of_ops_separable(mask_len, width, height)
  result = (2 * one_d_conv_ops) + (width*height)
  return result



DoG_number_of_ops_exploit(mask_len, image_x, image_y)

## Aggloerative H Clustering



In [ ]:
# Arguments - CENTRIOD NOT AVAILABLE - do by hand! :(

number_of_cluster = 3

C_FEATURES = [
    [5,10,20],
    [15,5,5],
    [10,20,15],
    [5,5,20],
    [15,15,15],
    [20,14,10],          
]
# 'complete' (maximum distance)
# 'average' (average distances between clusters) (THIS IS NOT CENTROID)
# 'single' (minimum distance)
link = 'average'
# 'manhatten' for SAD, or 'euclidean'  --> if its something else oh well.
aff='manhattan'




# EXTRA

# IF YOU DONT KNOW THE NUMBER OF CLUSTERS --> or want to see break own
metric = 'cityblock'  # 'euclidean'
# same as above 'centroid' (centroid)
# cant use SAD/cityblock with centroid, must be eucidean
link_method= 'average'


In [ ]:
# Standard clustering
def clustering(X,num, aff, link):
  cluster = AgglomerativeClustering(n_clusters=num, affinity=aff, linkage=link)
  result = cluster.fit_predict(X)
  return result

clustering(C_FEATURES, number_of_cluster, aff, link)


In [ ]:
# Prints dendogram
def clustering2(X,method, metric):
  Z = linkage(X, method=method, metric=metric)
  fig = plt.figure(figsize=(25, 10))
  dn = dendrogram(Z)
  plt.show()


clustering2(C_FEATURES, link_method, metric)

In [ ]:
# Computing Just Linkage Distances between two clusters
# Compute Distances between each feature in clusterA and clusterB
'''
    | B1  | B2 | B...
A1  |     |    |
.
.
.
AN
'''
# Then calculate for each linkage type. 
ALL_FEATURES = [
[1, 2, 4, 2], [2, 3, 2, 2], [1, 2, 1, 0], [1, 5, 4,1 ]      
]

all_similiarities(ALL_FEATURES, SAD)

## K Means Helper / Similarity between vectors

In [ ]:
# Computes similarities:
# K means Done mostly by by hand :/

'''
E.g.
given R = [[]], G = [[]], B = [[]]
and n init_clsuter with three elements each = [3].

Compute distance to all pixels
(e.g. top left pixel is the top left in R,G and B)

Select min distance to assign. 
'''

K_ALL_FEATURES = [
      [2, 1, 2, 1],
      [2, 3.5, 0.5, 2],
      [0.5, 0.75, 3.5, 1]        
]
# euclidean, SAD, cross_correlation, normalised_cross_correlation, correlation_coefficient
s_metric = normalised_cross_correlation

# similarirty of K_ALL_FEATURES[0]
# will be in row 0 and col 0 matchting to others

all_similiarities(K_ALL_FEATURES, s_metric)

## Pinhole camera & Thin Lens

In [ ]:
'''
Use  the  pinhole  camera  model  to  calculate  the  coordinates  (x’,y’)
of the  image  of  a  point  in  3D  space  which  has  coordinates (x,y,z)
relative to the opitical centre
'''
# Arguments
# x′=f′x/z
# y′=f′y/z

# ALL MUST BE SAME unit

f = 30  # mm
x = 400
y = 500
z = 6000


In [ ]:
def calculate_updated_x(f,x,z):
  return f * x / z

def calculate_updated_y(f,y,z):
  return f * y / z

def projected_pair(f,x,y,z):
  x_prime = calculate_updated_x(f,x,z)
  y_prime = calculate_updated_y(f,y,z)
  return (x_prime, y_prime)

projected_pair(f,x,y,z)

In [ ]:
# THIN LENS
# 1/f = 1/||z|| + 1/||z'||
#  1/||z'|| = 1/f - 1||z||

def thin_lens(f,z):
  # return z' - depth of image plane to make object in focus.
  return (1/f - 1/z) ** -1

thin_lens(f,z)

## Magnification Factors & Locations/Ref

In [ ]:
# MAGNINFICATION

# Arguments -mm
# α=−f′/sx
# β=−f′/sy
# CCD 

f = 20    # image plane distance

CCD_x = 12
CCD_y = 12

pixels_x = 800
pixels_y = 600


In [ ]:
def pixel_size_by_direction(direction, pixels):
  return direction/pixels

def magnification_factor(f,s):
  return -f/s

def magnification_factors(CCDx, CCDy, f, x_pixels, y_pixels):
  Sx = pixel_size_by_direction(CCDx, x_pixels)
  Sy = pixel_size_by_direction(CCDy, y_pixels)

  a = magnification_factor(f, Sx)
  B = magnification_factor(f, Sy)

  return (a,B)

magnification_factors(CCD_x, CCD_y, f, pixels_x, pixels_y)

In [ ]:
# LOCATION - to compute U,V

a = 925
B = 740

# Usually the center (so x_pixels/2, y_pixels/2)
Ox = 244
Oy = 180

# x,y z values
x = -10
y = 60
z = 650

def get_image_ref(alpha, beta, ox, oy, x, y, z):
    
    proj_op = np.array([[1,0,0,0],[0,1,0,0],[0,0,1,0]])
  
    cam_params = np.array([[alpha,0,ox],[0,beta,oy],[0,0,1]])

    coord = np.array([[x],[y],[z],[1]])

    ans = np.matmul(np.matmul(cam_params, proj_op),coord)
    print("REMEMBER TO ROUND NORMALLY (unless its for cross-ratio)! <5 ROUND DOWN, ≥5 ROUND UP")
    return (1/z) * ans

get_image_ref(a, B, Ox, Oy, x, y, z)

## Correspondence Depth From Movement Along X axis (perpendicular to optical axis)

In [ ]:
# Arguments
# 1000mm = 1m, 1mm = 0.001

time1 = 1
time2 = 1.04

# Focal length (NOTE: meters)
f = 0.035

# pixel size of the camera (NOTE: meters/pixel)
pixel_size = 0.0001

# camera movement (NOTE: meters/second)
movement = 0.5

# Corresponding points. (x,y)
I1_loc = (110,50)
I2_loc = (95,50)


In [ ]:
def x_move_depth(f,movement,p1, p2, t1, t2, size=1):
  '''
    Z= − fVx/ ̇x
    returns meters
  '''

  t_diff = t2-t1
  x_diff = p2[0]-p1[0]

  velocity_pixel = x_diff / t_diff    # pixels /second

  x_dot = size * velocity_pixel

  Z = - f * movement / x_dot 

  return Z

# May need to ignore rounding errors
x_move_depth(f, movement, I1_loc, I2_loc, time1, time2, pixel_size)

## Correspondence Depth, Camera Moves Along Z axis (along optical axis)

In [ ]:
# Arguments
# 1000mm = 1m, 1mm = 0.001

time1 = 10.2
time2 = 10.3

# centre coordinates
centre = (3, 3)

# camera movement (NOTE: meters/second)
movement = 0.2

# size, if not mentioned = 1
pixel_size=0.00015

# Corresponding points. (x,y)
I1_loc = (3,3)
I2_loc = (2,3)

In [ ]:
def z_move_depth(centre, movement,p1, p2, t1, t2,size=1):
  '''
    Z = xVz / ̇x
    returns meters
  '''

  t_diff = t2-t1
  p1_centre_x_diff = p1[0] - centre[0]
  p2_centre_x_diff = p2[0] - centre[0]
  x_diff = p2_centre_x_diff - p1_centre_x_diff

  velocity_pixel = x_diff / t_diff  # pixels / second

  x_dot = size * velocity_pixel

  Z = p1_centre_x_diff * movement / x_dot 

  return Z

# May need to ignore rounding errors, size is 1
z_move_depth(centre, movement, I1_loc, I2_loc, time1, time2)

## Time To Collision (no depth recovery)

In [ ]:
# Arguments
# 1000mm = 1m, 1mm = 0.001
# Assumes constant camera velocity.
time1 = 1
time2 = 1.04

# centre coordinates
centre = (100, 100)

# size, if not mentioned = 1
pixel_size=1

# Corresponding points. (x,y)
T1_loc = (140,100)
T2_loc = (145,100)

def time_to_collision(centre,p1, p2, t1, t2,size=1):
  '''
    T =  x1 / ̇x
    returns seconds
  '''

  t_diff = t2-t1
  p1_centre_x_diff = p1[0] - centre[0]
  p2_centre_x_diff = p2[0] - centre[0]
  x_diff = p2_centre_x_diff - p1_centre_x_diff

  velocity_pixel = x_diff / t_diff  # pixels / second

  x_dot = size * velocity_pixel

  T = p1_centre_x_diff / x_dot 

  return T

time_to_collision(centre, T1_loc, T2_loc, time1, time2)

## Image Diff, Sub, Average

not done but check lec 8, q7 (havent seen in exam before)

## Depth From Coplanar with collinear X axis (No Time)

In [ ]:
# Depth of Right Image (= Depth Left image).

# Arguments
# 1000mm = 1m, 1mm = 0.001

# Corresponding points have same y-axis (epipolar constraint)

# Focal length meters
f = 0.03    

# B - displacement of 2nd camera - meters
B = 0.4

# pixel size of the camera
size = 0.0001

left_loc = (231,345)
right_loc = (45, 345)


In [ ]:
def coplanar_x_colinear(f,B, left_point,right_point,size=1):
  '''
    Z=fB / xL−xR

    returns meters
  '''
  fB = f * B

  x_diff = left_point[0] - right_point[0]

  return fB / (size * x_diff)


coplanar_x_colinear(f, B, left_loc, right_loc, size)

## Hough Transform

In [ ]:
# Arguments

'''
e.g. matrix = [
  [1, 0],
  [1, 1]
]
# start from 0,0 where (col,row)
EG: positive_locations - [(0,0), (0,1), (1,1)]

results will show in order of angles
not really sure what table means but select the one with highest number
'''
R = 4

# R will not be greater than the length of the diagonal (i.e√(32+ 22) = 3.6, 
# so roundingup, maximum absolute value of r is 4)

positive_locations = [(0,0), (1,1), (2,2)]
angles = [0, 45, 90, 135]


In [ ]:
def r_calculate(theta, x, y):
  r = y * np.cos(np.deg2rad(theta)) - x * np.sin(np.deg2rad(theta))
  return int(round(r))

def perform_hough(positives, angles, R):

  ACC = {r: [0]*len(angles) for r in range(R,-(R+1),-1) }

  for x, y in positives:
    for i, t in enumerate(angles):
      r_key = r_calculate(t,x,y)
      ACC[r_key][i]+=1

  print(" "*2 + str(angles))
  for k, v in ACC.items():
    print(str(k) + ": " + str(v))

perform_hough(positive_locations, angles, R)

  [0, 45, 90, 135]
4: [0, 0, 0, 0]
3: [0, 0, 0, 0]
2: [1, 0, 0, 0]
1: [1, 0, 0, 0]
0: [1, 3, 1, 1]
-1: [0, 0, 1, 1]
-2: [0, 0, 1, 0]
-3: [0, 0, 0, 1]
-4: [0, 0, 0, 0]


## HARRIS

In [ ]:
# As seen on 16-17 5d
'''
(i) R≈0 occurs where intensity values are unchanging
(ii) R<0 occurs at edges
(iii) R>0 occurs at corners.
'''
# Arguments

# Image intentisities in x and y directions

k = 0.05
window = 3

Ix = [
      [1, 0, -2], 
      [0, -1, 0],
      [0, -3, 1],
]

Iy = [
       [2, 1, -3],
       [0, 0, 0],
       [0, -2, 1],
]


In [ ]:
def sum_patch(m, axis):
  return np.sum(m,axis=axis)


def harris_R_measure(Ix, Iy, k, window):
  #  R=[∑I2x∑I2y−(∑IxIy)2]−k[∑I2x+∑I2y]2

  Ix2 = np.square(Ix)
  # print(Ix2)
  Iy2 = np.square(Iy)
  # print(Iy2)
  IxIy = np.multiply(Ix,Iy)
  # print(IxIy)
  Ix2_patches = get_matrix_patches(Ix2, window, True)
  Iy2_patches = get_matrix_patches(Iy2, window, True)
  IxIy_patches = get_matrix_patches(IxIy, window, True)
  
  # 0, 3
  axis = (2,3)

  SUM_Ix2 = sum_patch(Ix2_patches, axis)
  SUM_Iy2 = sum_patch(Iy2_patches, axis)
  SUM_IxIy = sum_patch(IxIy_patches, axis)

  a = np.multiply(SUM_Ix2, SUM_Iy2)
  c = np.subtract(a, np.square(SUM_IxIy))

  d = -k * np.square(np.add(SUM_Ix2, SUM_Iy2))

  R = np.add(c,d)

  return R

harris_R_measure(Ix,Iy, k, window)

## Compare template To Matrix

In [ ]:
# will compare a patch (from one image), 
# against all patches in a matrix (another image)
# using some metric

# arguments

template = [
    [0, 1, 0, 0],
    [1, 0, 0, 0],
    [0, 1, 0, 1],
    [1, 0, 1, 0],
]

image_matrix = [
    [0, 0, 0, 0],
    [1, 0, 1, 0],
    [1, 1, 1, 0],
    [0, 1, 0, 0],
]

# from functions at the top:
# euclidean, SAD, cross_correlation, normalised_cross_correlation, correlation_coefficient
similairty_func = cross_correlation

# Set to true if you want 0 padded.
padded = False


In [ ]:
def metric_across_patches(image, template, metric, pad):
  template = np.array(template)
  t_rows, _ = template.shape
  flatten_template = template.flatten()
  patches = get_matrix_patches(image,t_rows, pad)
  rows, cols, _, __ = patches.shape
  result = []
  for r in range(rows):
    row_sim = []
    row_patches = patches[r]
    for p in row_patches:
      compare = [flatten_template, p.flatten()]
      c_res = pdist(compare, metric=metric)
      row_sim.append(c_res[0])
    result.append(row_sim)

  result = np.array(result)
  return result

print('NOTE BE CAREFUL - exam may require 1-index in (col,row)')
print('BE CAREFUL WITH NUMPY WHOLE NUMBERS, >5 0\'s, 9\'s round appropriately  ')
metric_across_patches(image_matrix, template, similairty_func, padded)

### Edge Matching Helper

Tutorial 9 q4


Average of the minimum distances, where min distance is euclidean. 

'closest distance from value in template position to same value position in image patch'   

### Number Of Operations

Tutorial 9, q2. - Normalised Cross Correlations 

In [ ]:
# total number of templates * number of patches * opperations per 1

# This method helps you calculate number of patches
pad_image = False
height = 200
width = 300

# 3x3 kernel has size 3
kernel_size = 11


In [ ]:
def number_of_patches(h, w, pad, size):
  image = np.zeros((h,w))
  matrix_patches = get_matrix_patches(image, size, pad)
  rows,cols,_, __ = matrix_patches.shape

  return rows*cols

number_of_patches(height, width,pad_image, kernel_size)

## Bayes Helper

exam 17-18 6d, tutorial 10, q9

In [ ]:
# Arguments

#  p(A | I) = p(I | O) * p(O) / p(I)
# posterier = likelihood * prior / evidence

# p(A)
pA = 0.8  # p(A)

# Assumed that P(I) = 1
pI = 1

pIgA = 0.02   #p(I | A)



In [ ]:
def p_A_given_I(pIgA, pA, pI):
  return (pIgA * pA) / pI

p_A_given_I(pIgA, pA, pI)